In [22]:
import random
from datetime import datetime, timedelta
import json
import os

dir_files_db = "."

filepath_items = os.path.join(dir_files_db, "db", "mock_data", "items.json")
filepath_customers = os.path.join(dir_files_db, "db", "mock_data", "customers.json")
filepath_transactions = os.path.join(
    dir_files_db, "db", "mock_data", "transactions.json"
)


# Helper function to generate random dates for history
def random_date(start_date, end_date):
    time_delta = end_date - start_date
    random_days = random.randint(0, time_delta.days)
    return start_date + timedelta(days=random_days)


today = datetime.now()
six_months_ago = today - timedelta(days=180)
one_year_ago = today - timedelta(days=365)

with open(filepath_customers, "r", encoding="utf-8") as file:
    customers = json.load(file)

with open(filepath_items, "r") as file:
    items: dict = json.load(file)

customers_new = {}
for n, customer in enumerate(customers.values()):
    customer["cid"] = n
    customers_new[n] = customer

customers_new_keys = list(customers_new.keys())

items_new = {}
for n, item in enumerate(items.values()):
    item_new = {
        "iid": n,
        "seller": random.choice(customers_new_keys),
        "name": item["name"],
        "category": item["category"],
        "brand": item["brand"],
        "quantity": random.randint(1, 100),
        "prices": item["prices"],
        "rating": round(random.uniform(1.0, 5.0), 2),
    }
    items_new[n] = item_new

with open(filepath_items, "w") as file:
    json.dump(items_new, file, indent=4)

with open(filepath_customers, "w") as file:
    json.dump(customers_new, file, indent=4)

with open(filepath_transactions, "w") as file:
    transactions = {}
    for n in range(10_000):
        item_id = random.choice(list(items_new.keys()))
        item = items_new[item_id]
        while True:
            customer_id = random.choice(customers_new_keys)
            if customer_id != item["seller"]:
                break

        transaction_date = random_date(
            datetime.fromisoformat(item["prices"][0]["timestamp"]), today
        )
        price = item["prices"][-1]["price"]
        for price_ in item["prices"]:
            if transaction_date >= datetime.fromisoformat(price_["timestamp"]):
                price = price_["price"]
                break
        quantity = random.randint(1, 5)
        transactions[n] = {
            "tid": n,
            "item_id": item_id,
            "buyer": customer_id,
            "date": transaction_date.strftime("%Y-%m-%dT%H:%M:%S"),
            "quantity": quantity,
            "price": price,
            "price_total": round(price * quantity, 2),
        }

    json.dump(transactions, file, indent=4)